In [44]:
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
from collections import OrderedDict
from nltk.corpus import stopwords
from collections import Counter
import numpy as np
import pandas as pd
import math
import nltk
import operator
import json

In [14]:
# baixando stopwords em portugues

nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /Users/rpss/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/rpss/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [49]:
# mock do array invertido
# segundo Ramom o indice sera composto por TERMO, DOCUMENTO, FREQUENCIA
# index = OrderedDict([
#     ('Blusa',[('DOCUMENTO1',2),('DOCUMENTO3',3)]),
#     ('Comprida',[('DOCUMENTO3',3),('DOCUMENTO4',1)]),
#     ('Longa',[('DOCUMENTO4',1),('DOCUMENTO1',1)]),
#     ('Cropped',[('DOCUMENTO4',1),('DOCUMENTO3',4)]),
#     ('Azul',[('DOCUMENTO3',2),('DOCUMENTO1',3)]),
#     ('Verde',[('DOCUMENTO4',1),('DOCUMENTO3',1)]),
# ])
index = json.load(open('./compress.json'), object_pairs_hook=OrderedDict)
consultaOriginal = "Blusas brancas e Longa Verde com Azul Azul"
len(index.items())
for term in index.items():
    print(term[0])

0.description
0.sizes
0024.description
0039.description
012.title
02.description
0362.description
04.description
05.description
074.title
0782.description
092.title
0px.description
1.description
1.sizes
10.description
100.description
12.description
12.sizes
12m.sizes
13.description
14.description
18.sizes
18m.sizes
1q.price
2.description
2.sizes
2.title
20.description
24m.sizes
2q.price
3.description
3.sizes
30.description
32.description
34.sizes
35.sizes
36.description
36.sizes
37.sizes
38.description
38.sizes
39.sizes
3m.sizes
3q.price
4.description
4.sizes
40.sizes
42.description
42.sizes
44.description
44.sizes
46.sizes
4q.price
5.description
5.sizes
50.description
6.description
6.sizes
6m.description
6m.sizes
7.description
7.title
70.description
73.description
8.description
8.title
88.description
9.sizes
90.description
93.description
94.description
96.description
97.description
9m.sizes
abaixo.description
abe.title
aberta.description
aberto.description
aberto.title
abertura.descri

In [50]:
def consulting(text):
    My_stopwords = set(stopwords.words('portuguese'))
    palavras = word_tokenize(text)
    lista_Termos = [i for i in palavras if i not in My_stopwords]
    return lista_Termos

# exemplo de consulta do usuario
display(consulting('Blusas Longas de cor branca'))


['Blusas', 'Longas', 'cor', 'branca']

In [51]:
# precisamos saber quais Termos vou buscar dentro do grande index, serão apenas Termos que
# estão presentes na minha consulta, vamos então criar um array somente de termos que existem no
# meu discionario de termos

def postingTerms(fraseInput, index):
    indexSize = len(index)
    consulta = consulting(fraseInput)
#     print(consulta)
    pesquisa = []
#     display(fraseInput)
#     display(consulta)
    index.items()
#     for term, postings in index.items():
#             index[term] = compress_postings(postings)
    #falta trazet tudo pro min
    for fraseInput in consulta:
        for term in index.items():
            if term[0] in fraseInput:
                if term[0] not in pesquisa:
#                     print(term[0],"está dentro")
                    termo = str(term[0])
                    pesquisa.append(termo)
    print("pesquisa deve ter", pesquisa)
    return pesquisa

pesquisa = postingTerms(consultaOriginal, index)

# similarmente precisando saber quais Documentos vou buscar dentro do grande index

def documentNames(pesquisa,index):
    listaNomes =[]
    for doc in index.items():
        for termo in pesquisa:
            if termo in doc[0]:
                if doc[1][0][0] not in listaNomes:
                    listaNomes.append(doc[1][0][0])
#         print("documentos da pesquisa", listaNomes)
    return listaNomes
listaNomes = documentNames(pesquisa,index)

pesquisa deve ter []


In [52]:
def matrixFrequencia(pesquisa,docs,index):
    matrix = np.zeros((len(pesquisa),len(docs)),dtype=object)
    #vamos criar um dataframe bonito que englobe os termos e os documentos da consulta 
    df = pd.DataFrame(matrix, columns=docs, index=pesquisa)
    
    # Document-at-time olhando se o termo está dentro de um documento e por fim preenchendo os docs, sem trabalhar com frequência.
    for documento in docs:
        for termo in pesquisa:
#             if termo in pesquisa:
#                 df.loc[termo,documento]=1
#             else:
#                 df.loc[termo,documento]=0
            for post in index.items():
                if termo in post[0]:
                    if documento in post[1][0][0]:
                        df.loc[termo,documento] =1
#     display("depois analise",df)
    return df

def matrixFrequenciaTFIDF(pesquisa,docs,index):
    docs.insert(0, "Times")
#     print(docs)
    matrix = np.zeros((len(pesquisa),len(docs)),dtype=object)
#     print(matrix)
    #vamos criar um dataframe bonito que englobe os termos e os documentos da consulta 
    df = pd.DataFrame(matrix, columns=docs, index=pesquisa)
#     display("antes da analise",df)
    
    # Document-at-time olhando se o termo está dentro de um documento e por fim preenchendo os docs, sem trabalhar com frequência.
    for documento in docs:
        for termo in pesquisa:
            if termo in pesquisa and documento == "Times":
                 #queremos levar em consideração a frequencia de pesquisa
                searchTimes = consultaOriginal.count(termo)
                df.loc[termo,documento] = searchTimes
            else:
                df.loc[termo,documento]=0
            for post in index.items():
                if termo in post[0]:
                    if documento in post[1][0][0]:
                        if post[1][0][1] > 0:
                            df.loc[termo,documento] =post[1][0][1]
                        else:
                            df.loc[termo,documento] =0
#     display("depois analise",df)
    return df

In [53]:
#exemplo de matrix Frequencia usando TFIDF
matrixFrequenciaTFIDF(pesquisa,listaNomes,index)

,Times


In [54]:
#exemplo de matrix Frequencia SEM TFIDF
matrixFrequencia(pesquisa,listaNomes,index)

,Times


In [55]:
# Similaridade Cosseno para medir o score
def cosine_similarity(v,w):
    similirity = np.dot(v,w)/(math.sqrt((np.dot(v,v))*(np.dot(w,w))))
#     display(similirity)
    return similirity

#trabalhando com respostas SEM
def scoreThis(matrix):
    documentsList = []    
    for docName in matrix.columns:  
        score = cosine_similarity(1, matrix[docName])
        score=max(score)
        documentsList.append((docName,score))
    documentsList.sort(key=operator.itemgetter(1),reverse=True)
    display(documentsList)
    return documentsList


def scoreThisTDIDF(matrix):
    documentsList = []
    timesList = matrix['Times']
    for docName in matrix.columns:  
        score = cosine_similarity(timesList, matrix[docName])
        documentsList.append((docName,score))
    documentsList.sort(key=operator.itemgetter(1),reverse=True)
    return documentsList

In [56]:
# exemplo da pesquisa sem usar TFIDF
pesquisa = postingTerms("Quero Blusa Comprida bem Longa, Azul com cropped Verde", index)
listaNomes = documentNames(pesquisa,index)
matrix = matrixFrequencia(pesquisa,listaNomes,index)
scoreThis(matrix)

pesquisa deve ter []


[]

[]

In [57]:
# exemplo da pesquisa usando TFIDF
pesquisa = postingTerms("Quero Blusa Comprida bem Longa, Azul com cropped Verde", index)
listaNomes = documentNames(pesquisa,index)
matrix = matrixFrequenciaTFIDF(pesquisa,listaNomes,index)
scoreThisTDIDF(matrix)

pesquisa deve ter []


TypeError: unsupported operand type(s) for *: 'NoneType' and 'NoneType'

In [39]:
## correlação de kendle tau

pesquisa2 = postingTerms("Gostaria de uma Blusa Azul longa com um Cropped", index)
listaNomes = documentNames(pesquisa,index)
matrix = matrixFrequencia(pesquisa,listaNomes,index)
scoresR1 = scoreThis(matrix)

pesquisa2 = postingTerms("Gostaria de uma Blusa Azul longa com um Cropped  ", index)
listaNomes2 = documentNames(pesquisa2,index)
matrix2 = matrixFrequenciaTFIDF(pesquisa2,listaNomes2,index)
scoresR2 = scoreThisTDIDF(matrix2)

pesquisa deve ter ['Blusa', 'Azul', 'Cropped']


[('DOCUMENTO1', 1.0),
 ('DOCUMENTO3', 0.7071067811865475),
 ('DOCUMENTO4', 0.7071067811865475)]

pesquisa deve ter ['Blusa', 'Azul', 'Cropped']


In [40]:
display("CONSULTA SEM TFIDF")
display(scoresR1) 
display("CONSULTA COM TFIDF")
display(scoresR2)

'CONSULTA SEM TFIDF'

[('DOCUMENTO1', 1.0),
 ('DOCUMENTO3', 0.7071067811865475),
 ('DOCUMENTO4', 0.7071067811865475)]

'CONSULTA COM TFIDF'

[('Times', 1.0),
 ('DOCUMENTO3', 0.8944271909999159),
 ('DOCUMENTO1', 0.4472135954999579),
 ('DOCUMENTO4', 0.0)]

In [43]:
scoresR1[1][0]

'DOCUMENTO3'